# Bank account data exploration (via BBVA's monthly reports)

* Download the monthly reports via web. 
* Save all the reports in YYYY-MM.pdf format.
* The loading method expects the reports to be in a reports folder.

## Loading stuff

In [ ]:
import pdftotext, re, os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import matplotlib.dates as mdates

plt.style.use('seaborn')
plt.rcParams['figure.dpi'] = 400
plt.rcParams['savefig.dpi'] = 800
plt.rcParams["axes.grid"] = True

DIRNAME = 'reports'

COLUMN_NAMES = ['date', 'value_date', 'concept', 'amount', 'balance', 'card', 'subconcept']

REGEX = r'^([\d][\d]/[\d][\d])\s([\d][\d]/[\d][\d])\s*([A-ZÑñÁáÉéÍíÓóÚúÜü\'\,\.\:\s]+)\s*(-?\d*.?\d*,\d*)\s*(\d*.?\d*,\d*)\s*(\d*)\s*([\d\wÑñÁáÉéÍíÓóÚúÜü \.\,\:\*\'\-\/\(\)]*)$'

parse_num = lambda row: pd.to_numeric(row.apply(lambda x: x.replace('.', '').replace(',', '.')))
parse_date = lambda row, year: pd.to_datetime(row + '/' + year, dayfirst=True)
parse_text = lambda row: row.apply(lambda x: x.strip())

def parse_pdf(filename):
    year = filename[:4]
    with open(os.path.join(DIRNAME, filename), "rb") as f:
        pdf = '\n'.join(pdftotext.PDF(f))
        operations = re.findall(REGEX, pdf, re.MULTILINE)
        df = pd.DataFrame(operations, columns = COLUMN_NAMES)
        df['date'] = parse_date(df['date'], year)
        df['value_date'] = parse_date(df['value_date'], year)
        df['amount'] = parse_num(df['amount'])
        df['balance'] = parse_num(df['balance'])
        df['concept'] = parse_text(df['concept'])
        df['subconcept'] = parse_text(df['subconcept'])
        return df

filenames = [ f for _, _, files in os.walk(DIRNAME) for f in files]
df = pd.concat(map(parse_pdf, filenames))
df.set_index('date', inplace=True) 
df.sort_index(inplace=True)
df.describe()

## Common routines

In [ ]:
def group_by_month(dataframe, aggregation_dict):
    grouped_dataframe = dataframe.groupby(pd.Grouper(freq='M')).aggregate(aggregation_dict)
    grouped_dataframe.index = grouped_dataframe.index.map(lambda x: x if isinstance(x, str) else x.strftime('%Y-%m'))
    return grouped_dataframe

## Balance evolution

### Overall evolution

In [ ]:
df_last_year = df.last('12M')

_, (ax1, ax2) = plt.subplots(nrows=2, ncols=1, figsize=(10, 20))

df.balance.plot(ax=ax1)
df_last_year.balance.plot(ax=ax2)

### Monthly evolution in mean terms

In [ ]:
balance_by_month_mean = group_by_month(df, {'balance': np.mean})

_, (ax1, ax2) = plt.subplots(nrows=2, ncols=1, figsize=(10, 10))

balance_by_month_mean.plot(marker='o', legend=False, ax=ax1)
balance_by_month_mean.tail(24).plot(marker='o', legend=False, ax=ax2)

### Monthly evolution in diff terms

In [ ]:
def diff(series):
    aslist = series.tolist()
    return aslist[-1] - aslist[0] if len(aslist) != 0 else 0

balance_by_month_diff = group_by_month(df, {'balance': diff})

_, (ax1, ax2) = plt.subplots(nrows=2, ncols=1, figsize=(10, 10))

balance_by_month_diff.plot(marker='o', legend=False, ax=ax1)
balance_by_month_diff.tail(24).plot(marker='o', legend=False, ax=ax2)

## Spending vs incoming

### Spending vs incoming by concepts

In [ ]:
spending = df.query('amount < 0')
incoming = df.query('amount > 0')
spending_by_concept = spending.groupby('concept').amount.sum()
incoming_by_concept = incoming.groupby('concept').amount.sum()

combined_amounts = pd.concat([spending_by_concept.rename('spending'), incoming_by_concept.rename('incoming')], axis=1)

combined_amounts.plot(kind='barh', width=1, figsize=(10,10), stacked=True)

### Last year spending vs incoming

In [ ]:
spending_last_year = df_last_year.query('amount < 0')
incoming_last_year = df_last_year.query('amount > 0')

spending_last_year_by_month = group_by_month(spending_last_year, {'amount':np.sum}).amount
incoming_last_year_by_month = group_by_month(incoming_last_year, {'amount':np.sum}).amount

combined_amounts_last_year_by_month = pd.concat([spending_last_year_by_month, incoming_last_year_by_month], axis=1)

ax = balance_by_month_mean.tail(12).plot(legend=False, linestyle='-', marker='o', color='crimson')
ax = balance_by_month_diff.tail(12).plot(legend=False, linestyle='-', marker='o', color='goldenrod', ax=ax)
ax = combined_amounts_last_year_by_month.plot(kind='bar', stacked=True, legend=False, ax=ax)
ax.legend(['mean', 'diff']);

## Spending distribution

### Spending distribution by €

In [ ]:
_, (ax1, ax2) = plt.subplots(nrows=1, ncols=2)

spending.amount.abs().plot(kind='hist', bins=25, xlim=(0, 1000), ax=ax1)
spending.amount.abs().plot(kind='box', ylim=(0,150), yticks=range(0, 150, 10), ax=ax2)

### Spending by concept

In [ ]:
spending_by_concept_sorted = spending_by_concept.abs().sort_values()

spending_by_concept_sorted.where(lambda x : x > 500).plot(kind='barh')

### Spending by month

In [ ]:
spending_by_month = group_by_month(spending, {'amount': np.sum}).abs()


spending_by_month.amount.plot(kind='bar', width=1, figsize=(20, 20))